# Demo: Knapsack Problem

In this notebook, we demonstrate how to solve the knapsack problem, a classic Operations Research problem. In this problem, we have a knapsack with fixed weight capacity and a set of items each associated with a weight and a value. We want fit items into the knapsack in a way such as the total value of all fitted items is as high as possible, however, the sum of item weights cannot exceed the knapsack weight capacity. 

While is problem is traditionally solved with linear programming, we can convert it into a contextual bandit problem (a simplified 1-stage Markov decision problem) and solve it using pymdp.

Let us define our actions `a_i` as whether to include an item or not for each item i. The state `s_i` of the system is defined as whether an item is included or not, i.e., copying the action variables over to the corresponding state variables. We also need another state variable `z` which represents whether the knapsack capacity is exceeded. If an item is included, i.e., `s_i = 1`, we get a reward `r_i`, otherwise, we get a reward of 0 when `s_i = 0`. We can thus define our preference of including valuable items to be proportional to the exponential of reward: `C[s_i] = softmax([0, r_i])`. Our preference on the capacity constraint variable `z` is to never violate it, i.e., `C[z] = [1, 0]`. Since the system is fully observable, we will set all the Categorical/Dirichlet parameters of the emission model (the `A` tensors) to be diagonal.

In [4]:
import numpy as np
from jax import numpy as jnp, random as jr
from jax import tree_util as jtu
import jax.nn as nn

from pymdp.agent import Agent
from pymdp import distribution

## Specify model structure

In [10]:
# knapsack problem setup
num_items = 5
max_capacity = 20
item_weights = max_capacity / num_items + np.random.uniform(0, 5, size=(num_items,))
rewards = jr.uniform(jr.PRNGKey(0), shape=(num_items,), minval=0.0, maxval=1.0)

print("item rewards", rewards)
print("item weights", item_weights)
print("item weight sum", item_weights.sum())

# mdp config
state_config = {
    f"s_{i}": {"elements": ["not enclude", "include"], "depends_on": [f"s_{i}"], "controlled_by": [f"a_{i}"]} 
    for i in range(num_items)
}
state_config["z"] = {
    "elements": ["not violated", "violated"], "depends_on": ["z"], "controlled_by": [f"a_{i}" for i in range(num_items)]
} 

obs_config = {
    k: {"elements": v["elements"], "depends_on": [k]} for k, v in state_config.items()
}

act_config = {
    f"a_{i}": {"elements": ["not enclude", "include"]} 
    for i in range(num_items)
}

model_description = {
    "observations": obs_config,
    "controls": act_config,
    "states": state_config,
}

item rewards [0.947667   0.9785799  0.33229148 0.46866846 0.5698887 ]
item weights [8.3257477  5.90736203 6.15180577 8.64391967 4.4631106 ]
item weight sum 33.491945771719365


## Specify model parameters

In [12]:
model = distribution.compile_model(model_description)

print("A shapes", [a.data.shape for a in model.A])
print("B shapes", [a.data.shape for a in model.B])

A shapes [(2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2)]
B shapes [(2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2, 2, 2, 2, 2)]


In [18]:
def create_identity_transition_factor(mat):
    for i in range(mat.shape[1]):
        mat[:, i] = np.eye(len(mat))
    return mat

def create_constraint_factor_z_greater_than(act_dim, maximum, num_items, weights):
    # Create an array of shape (2, act_dim, act_dim, ..., act_dim)
    tensor_shape = (2,) + (act_dim,) * num_items
    
    # Create an array with indices from 0 to act_dim - 1 along each dimension
    indices = np.indices(tensor_shape[1:])

    # Reshape weights to fit indices shape
    weights_reshaped = np.array(weights).reshape((-1,) + (1,) * (indices.ndim - 1))
    # Multiply weights with matrix that conforms to constraint
    result = np.array(indices == (act_dim - 1)) * weights_reshaped

    # Calculate the total for each combination of actions
    total = np.sum(result, axis=0)
    
    # Create the tensor based on the total hours condition
    tensor = np.where(total > maximum, 1, 0)

    # Stack the tensor along the first axis to create the final tensor
    tensor = np.stack((1 - tensor, tensor), axis=0)

    # make a copy for self state 
    tensor = np.stack([tensor, tensor], axis=1)
    return tensor

# update A tensor
for i in range(len(model.A)):
    model.A[i].data = np.eye(len(model.A[i].data))

# update B tensors
for i in range(num_items):
    model.B[i].data = create_identity_transition_factor(model.B[i].data)

model.B[-1].data = create_constraint_factor_z_greater_than(2, max_capacity, num_items, item_weights)

# create C tensors
preferences = nn.softmax(np.stack([np.zeros_like(rewards), rewards], axis=-1), axis=-1)
Cs = [None for _ in range(len(model.A))]
for i in range(len(model.A)):
    Cs[i] = preferences[i]
    
Cs[-1] = np.array([1., 0]) # capacity constraint cannot be violated

print("A shapes", [a.data.shape for a in model.A])
print("B shapes", [a.data.shape for a in model.B])

print("A normalized", [np.isclose(a.data.sum(0), 1.).all() for a in model.A])
print("B normalized", [np.isclose(a.data.sum(0), 1.).all() for a in model.A])
print("C normalized", [np.isclose(a.sum(0), 1.).all() for a in Cs])

A shapes [(2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2)]
B shapes [(2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2, 2, 2, 2, 2)]
A normalized [np.True_, np.True_, np.True_, np.True_, np.True_, np.True_]
B normalized [np.True_, np.True_, np.True_, np.True_, np.True_, np.True_]
C normalized [np.True_, np.True_, np.True_, np.True_, np.True_, np.True_]


## Run agent

In [23]:
B_action_dependencies = [
    [list(model_description["controls"].keys()).index(i) for i in s["controlled_by"]] 
    for s in model_description["states"].values()
]
num_controls = [len(c["elements"]) for c in model_description["controls"].values()]

agent = Agent(
    model.A, model.B, Cs,
    B_action_dependencies=B_action_dependencies,
    num_controls=num_controls,
)

In [24]:
qs = jtu.tree_map(lambda x: jnp.expand_dims(x, axis=0), agent.D)
q_pi, G = agent.infer_policies(qs)
action = agent.sample_action(q_pi)
action_multi = agent.decode_multi_actions(action)

In [25]:
print("best action", action)
print("best action multi", action_multi)
print("item weights")
print(item_weights)
print("item rewards")
print(rewards)

best action [[ 1  1  0  0  1 25]]
best action multi [[1 1 0 0 1]]
item weights
[8.3257477  5.90736203 6.15180577 8.64391967 4.4631106 ]
item rewards
[0.947667   0.9785799  0.33229148 0.46866846 0.5698887 ]


In [26]:
# compare actions
from pymdp import utils
for i, idx in enumerate(np.argsort(q_pi[0])[::-1]):
    action_multi_f = utils.index_to_combination(agent.policies[idx, 0][-1].tolist(), agent.num_controls_multi)
    print("\naction", agent.policies[idx, 0])
    print("action multi", action_multi_f)
    print("efe: {:.2f}, reward: {:.2f}".format(G[0, idx], np.sum(rewards * action_multi_f)))
    if i == 5:
        break


action [ 1  1  0  0  1 25]
action multi [1 1 0 0 1]
efe: 3.89, reward: 2.50

action [ 0  1  0  1  1 11]
action multi [0 1 0 1 1]
efe: 3.68, reward: 2.02

action [ 0  1  1  0  1 13]
action multi [0 1 1 0 1]
efe: 3.61, reward: 1.88

action [ 1  1  0  0  0 24]
action multi [1 1 0 0 0]
efe: 3.61, reward: 1.93

action [ 1  0  1  0  1 21]
action multi [1 0 1 0 1]
efe: 3.60, reward: 1.85

action [0 1 0 0 1 9]
action multi [0 1 0 0 1]
efe: 3.45, reward: 1.55
